In [2]:
#!pip install sentence-transformers faiss-cpu numpy


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss


In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print("Model loaded successfully")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\hp\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded successfully


In [15]:
documents = [
    "Machine learning is powerful",
    "Deep learning uses neural networks",
    "The cat is sleeping on the sofa",
    "Artificial intelligence is the future",
    "Dogs are loyal animals",
    "Transformers changed NLP forever"
]

print("Total documents:", len(documents))


Total documents: 6


In [16]:
embeddings = model.encode(documents)

print("Embedding shape:", embeddings.shape)


Embedding shape: (6, 384)


In [17]:
dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)  # Exact search using L2 distance

index.add(np.array(embeddings))

print("Total vectors stored in FAISS:", index.ntotal)


Total vectors stored in FAISS: 6


In [8]:
def search(query, k=2):
    query_embedding = model.encode([query])
    
    distances, indices = index.search(np.array(query_embedding), k)
    
    print("\nQuery:", query)
    print("\nTop Matches:")
    
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {documents[idx]} (Distance: {distances[0][i]:.4f})")


In [20]:
search("Dogs are loyal animals", k=3)



Query: Dogs are loyal animals

Top Matches:
1. Dogs are loyal animals (Distance: 0.0000)
2. The cat is sleeping on the sofa (Distance: 1.8799)
3. Deep learning uses neural networks (Distance: 1.9058)


In [21]:
# Test Failure (Random Vectors)
random_vectors = np.random.rand(len(documents), dimension).astype('float32')

random_index = faiss.IndexFlatL2(dimension)
random_index.add(random_vectors)

query_embedding = model.encode(["Dogs are loyal animals"])

distances, indices = random_index.search(np.array(query_embedding), 2)

print("Results with random vectors (should be meaningless):")
for idx in indices[0]:
    print(documents[idx])


Results with random vectors (should be meaningless):
Transformers changed NLP forever
Machine learning is powerful


In [22]:
# cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

query = "Artificial intelligence"

query_embedding = model.encode([query])

similarities = cosine_similarity(query_embedding, embeddings)

print("Cosine Similarity Scores:")
for doc, score in zip(documents, similarities[0]):
    print(f"{doc} --> {score:.4f}")


Cosine Similarity Scores:
Machine learning is powerful --> 0.5515
Deep learning uses neural networks --> 0.4103
The cat is sleeping on the sofa --> -0.0959
Artificial intelligence is the future --> 0.7053
Dogs are loyal animals --> 0.0863
Transformers changed NLP forever --> 0.1659
